In [1]:
from keras.optimizers import SGD, Nadam, Adam, RMSprop
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras.models import load_model


from keras.models import Sequential
from keras.layers import Dense, Activation


def buildModel(self):
        """Model layers"""
        
        char_size = 52
        # character input
        character_input = Input(shape=(None, char_size,), name="Character_input")
        embed_char_out = TimeDistributed(
            Embedding(len(self.char2Idx), 30, embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name="Character_embedding")(
            character_input)

        dropout = Dropout(self.dropout)(embed_char_out)

        # CNN
        conv1d_out = TimeDistributed(Conv1D(kernel_size=self.conv_size, filters=30, padding='same', activation='tanh', strides=1), name="Convolution")(dropout)
        maxpool_out = TimeDistributed(MaxPooling1D(char_size), name="Maxpool")(conv1d_out)
        char = TimeDistributed(Flatten(), name="Flatten")(maxpool_out)
        char = Dropout(self.dropout)(char)

        # word-level input
        words_input = Input(shape=(None,), dtype='int32', name='words_input')
        words = Embedding(input_dim=self.wordEmbeddings.shape[0], output_dim=self.wordEmbeddings.shape[1], weights=[self.wordEmbeddings],
                          trainable=False)(words_input)

        # case-info input
        casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
        casing = Embedding(output_dim=self.caseEmbeddings.shape[1], input_dim=self.caseEmbeddings.shape[0], weights=[self.caseEmbeddings],
                           trainable=False)(casing_input)

        # concat & BLSTM
        output = concatenate([words, casing, char])
        output = Bidirectional(GRU(self.lstm_state_size, 
                                    return_sequences=True, 
                                    dropout=self.dropout,                        # on input to each LSTM block
                                    recurrent_dropout=self.dropout_recurrent     # on recurrent input signal
                                   ), name="BLSTM")(output)
        
        #output = TimeDistributed(Dense(len(self.label2Idx), activation='softmax'),name="Softmax_layer")(output)

        # set up model
        #self.model = Model(inputs=[words_input, casing_input, character_input], outputs=[output])
        #self.model.compile(loss='sparse_categorical_crossentropy', optimizer=self.optimizer)
        
        
        output = TimeDistributed(Dense(50, activation="relu"),name="Time_Dense_Layer")(output)  # a dense layer as suggested by neuralNer

        # CRF Layer
        crf = CRF(len(self.label2Idx), sparse_target=True)
        out = crf(output)
        
        model = Model(inputs=[words_input, casing_input, character_input], outputs=[out])
        model.compile(optimizer=self.optimizer, loss=crf_loss, metrics=[crf.accuracy, crf_viterbi_accuracy])




model = build_model()
model.load_weights('my_weights.model')



# identical to the previous one
weigths = model.load_weights('crf.h5')

Using TensorFlow backend.


NameError: name 'model' is not defined